### Italian Stock Exchange: Web Scraping all the bonds

In [1]:
# import modules and functions
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from read_table import read_table_url_isin, read_table_url_selenium_driverless_xpath_nointestazione


In [2]:
now = datetime.now() # current date and time
date_time = now.strftime("%m-%d-%Y")
output_name='lista_bond_'+ date_time

# Find all the Bond ISIN on the italian Stock Market 
# Troviamo tutti i valori ISIN dei bond presenti su Borsa Italiana

url_isin='https://www.borsaitaliana.it/borsa/obbligazioni/advanced-search.html?page=&size=10000000&lang=it' #qui la cosa smart è mettere un valore altissimo dopo size
class_table='m-table -firstlevel'

#let's use the function read_table for getting the list of all bonds
# usiamo la funzione read_table
tables=read_table_url_isin(url_isin,class_table)

#in questo caso dovrebbe essere solo una la tabella restituita
df=tables[0]
print(df['Isin'].size)

1959


In [3]:
# let's opern a web driver that will be opened during the scraping of information about bonds
#apriamo un driver solo per abbassare il tempo di cariamento
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-software-rasterizer')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-extensions')
options.add_argument('--disable-infobars')
options.add_argument('--disable-notifications')
options.add_argument('--disable-crash-reporter')
options.add_argument('--disable-extensions')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(options=options)

# Let's open all the bonds pages, scraping with function read_table and saving all the values
# Apriamo una ad una tutte le pagine delle obbligazioni, runniamo read_table, e salviamo i valori di tutte le voci
isin_failed=[]
for index, row in df.iterrows():
    if index<df['Isin'].size: #test
        isin=row['Isin']
        print('Inizio isin: '+isin)
        url_scheda='https://www.borsaitaliana.it/borsa/obbligazioni/mot/euro-obbligazioni/scheda/'+isin+'.html?lang=it'
        xpaths=['//*[@id="fullcontainer"]/main/section/div[4]/div[5]/article/div/div[2]/div[1]/table',
                '//*[@id="fullcontainer"]/main/section/div[4]/div[5]/article/div/div[2]/div[2]/table',
                '//*[@id="fullcontainer"]/main/section/div[4]/div[6]/article[1]/div/div[2]/table',
                '//*[@id="fullcontainer"]/main/section/div[4]/div[8]/div/article/div/div[2]/div[1]/table',
                '//*[@id="fullcontainer"]/main/section/div[4]/div[8]/div/article/div/div[2]/div[2]/table']
        #se sezione "informazioni sostenibili" non è presente nella schermata
        xpaths_failed=['//*[@id="fullcontainer"]/main/section/div[4]/div[4]/article/div/div[2]/div[1]/table',
                '//*[@id="fullcontainer"]/main/section/div[4]/div[4]/article/div/div[2]/div[2]/table',
                '//*[@id="fullcontainer"]/main/section/div[4]/div[5]/article[1]/div/div[2]/table',
                '//*[@id="fullcontainer"]/main/section/div[4]/div[7]/div/article/div/div[2]/div[1]/table',
                '//*[@id="fullcontainer"]/main/section/div[4]/div[7]/div/article/div/div[2]/div[2]/table']

        tables,flag_failed=read_table_url_selenium_driverless_xpath_nointestazione(url_scheda,xpaths,driver)
        if flag_failed==1:
            print('Provo con secondo set di xpaths')
            tables,flag_failed=read_table_url_selenium_driverless_xpath_nointestazione(url_scheda,xpaths_failed,driver)

        if flag_failed==0:
            tabellone=pd.DataFrame()
            for table in tables:
                table=table.T
                tabellone=pd.concat([tabellone,table], axis=1)
            for column in tabellone.columns:
                df.loc[index,column]=tabellone[column].values[0]
        else:
            print('Fallito anche il secondo tentativo')
            isin_failed.append(isin)
        print(f"Avanzamento: {(index+1)/df['Isin'].size*100:.2f}%")

#Let's quit the driver and save the Excel file
#chiudiamo il driver
driver.quit()
#stampiamo in excel
df.to_excel(output_name+".xlsx")


Inizio isin: AT0000383864
Avanzamento: 0.05%
Inizio isin: AT0000A04967
Avanzamento: 0.10%
Inizio isin: AT0000A0DXC2
Avanzamento: 0.15%
Inizio isin: AT0000A0U299
Avanzamento: 0.20%
Inizio isin: AT0000A0VRQ6
Avanzamento: 0.26%
Inizio isin: AT0000A10683
Avanzamento: 0.31%
Inizio isin: AT0000A185T1
Avanzamento: 0.36%
Inizio isin: AT0000A1FAP5
Avanzamento: 0.41%
Inizio isin: AT0000A1K9C8
Avanzamento: 0.46%
Inizio isin: AT0000A1K9F1
Avanzamento: 0.51%
Inizio isin: AT0000A1PEF7
Avanzamento: 0.56%
Inizio isin: AT0000A1VGK0
Avanzamento: 0.61%
Inizio isin: AT0000A1XML2
Avanzamento: 0.66%
Inizio isin: AT0000A1Y3P7
Xpath 0 non trovato. flag_failed=1
Provo con secondo set di xpaths
Avanzamento: 0.71%
Inizio isin: AT0000A1ZGE4
Avanzamento: 0.77%
Inizio isin: AT0000A269M8
Avanzamento: 0.82%
Inizio isin: AT0000A28KX7
Avanzamento: 0.87%
Inizio isin: AT0000A2CQD2
Avanzamento: 0.92%
Inizio isin: AT0000A2EJ08
Avanzamento: 0.97%
Inizio isin: AT0000A2HLC4
Avanzamento: 1.02%
Inizio isin: AT0000A2KQ43
Avanzam